In [1]:
import pandas as pd
import json
import os

In [2]:
path = "D:\\phonepe_project\\pulse\\data\\map\\user\\hover\\country\\india\\state"
path_a = os.listdir(path)

In [ ]:
std4 = {'state':[], 'year':[],'quater':[],'district':[], 'registered_users':[], 'app_opens':[]}

for i in path_a:
    path_yr = path+"\\"+i+"\\"
    year = os.listdir(path_yr)

    for j in year:
        path_file = path_yr + j
        path_n = os.listdir(path_file)

        for k in path_n:
            asd = path_file + "\\" + k
            data_n = open(asd, "r")
            output = json.load(data_n)

            for l in output['data']['hoverDataList']:
                district = l['name']
                count = l['metric'][0]['count']
                amount = l['metric'][0]['amount']
                std4['transaction_type'].append(district)
                std4['transaction_count'].append(count)
                std4['transaction_amount'].append(amount)
                std4['state'].append(i)
                std4['year'].append(j)
                std4['quater'].append(int(k.strip('.json')))

df = pd.DataFrame(std4)

#not use this code

In [6]:
std4 = {'state': [], 'year': [], 'quater': [], 'district': [], 'registered_users': [], 'app_opens': []}

for i in path_a:
    path_yr = path + "\\" + i + "\\"
    year = os.listdir(path_yr)

    for j in year:
        path_file = path_yr + j
        path_n = os.listdir(path_file)

        for k in path_n:
            asd = path_file + "\\" + k
            with open(asd, "r") as data_n:
                try:
                    output = json.load(data_n)
                except json.JSONDecodeError as e:
                    print(f"Error loading JSON in {asd}: {e}")
                    continue

            if output and output.get('data') and output['data'].get('hoverData'):
                for district, data in output['data']['hoverData'].items():
                    registered_users = data.get('registeredUsers', 0)
                    app_opens = data.get('appOpens', 0)

                    std4['district'].append(district)
                    std4['registered_users'].append(registered_users)
                    std4['app_opens'].append(app_opens)
                    std4['state'].append(i)
                    std4['year'].append(j)
                    std4['quater'].append(int(k.strip('.json')))
            else:
                print(f"Missing or invalid data in {asd}")

df4 = pd.DataFrame(std4)


In [7]:
df4

,state,year,quater,district,registered_users,app_opens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
16831,west-bengal,2023,3,nadia district,1544294,12674328
16832,west-bengal,2023,3,birbhum district,974363,18961683
16833,west-bengal,2023,3,purba medinipur district,1530289,18593769
16834,west-bengal,2023,3,maldah district,1091634,28020285


In [8]:
df4.to_csv(r'D:\phonepe_project\map_user.csv')

In [9]:
import psycopg2

mydb = psycopg2.connect(
    host = "localhost",
    database = "postgres",
    user = "postgres",
    password = "123456")

mycursor = mydb.cursor()

In [11]:
query4 = """
            CREATE TABLE IF NOT EXISTS map_user(
                state VARCHAR(255),
                year VARCHAR(4),
                quater INT, 
                district VARCHAR(255), 
                registered_users INT, 
                app_opens INT
                )

            """ 

try:
    mycursor.execute(query4)
    mydb.commit()
    print("Tables created successfully.")
except Exception as e:
    print("Error:", e)
    mydb.rollback()

Tables created successfully.


In [12]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL database connection parameters
db_config = {
    'user': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')

# CSV file path
csv_file_path = 'D:\\phonepe_project\\map_user.csv'

# Read CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Insert data into PostgreSQL table
try:
    df.to_sql('map_user', con=engine, if_exists='replace', index=True)
    print("Data inserted successfully.")
except Exception as e:
    print("Error:", e)
finally:
    engine.dispose()  # Close the database connection


Data inserted successfully.
